**This notebook contains a script that assissts in the management of student documents, files and folders.**

**Import Libraries**

In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

**Authenticate Google Drive**

In [14]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Get File By ID**

In [2]:
#create method that returns files in a folder by id
def list_files(folder_id):
    file_list = drive.ListFile(
        {
            'q': "'{}' in parents and trashed=false".format(folder_id),
            'corpora': "teamDrive",
            'teamDriveId': "0ALGjY-PCeStEUk9PVA",
            'includeTeamDriveItems': "true",
            'supportsTeamDrives': "true"
        }
    ).GetList()
    return file_list

**Create Google Drive Folder**

In [3]:
def create_folder(parent_folder_id, folder_name):
    #create folder if it does not exist
    file_list = list_files(parent_folder_id)
    folder_id = ''
    for folder in file_list:
        if folder['title'] == folder_name:
            return folder['id']
    if not folder_id:
        folder = drive.CreateFile({'parents': [{'id': parent_folder_id}], 'title': folder_name,
                                   'mimeType': 'application/vnd.google-apps.folder'})
        folder.Upload()
        return folder['id']
    return False

**Copy Google Drive File**

In [4]:
def copy_file(folder_id, file_name, file_params):
    #download file
    files = list_files(file_params['parent_id'])
    target_file = files[0]
    for file in files:
        if file['id'] == file_params['file_id']:
            target_file = file

    target_file.GetContentFile(file_name)
    
    #upload file to new folder location
    file = drive.CreateFile({'parents': [{'id': folder_id}], 'title': file_name})
    file.SetContentFile(file_name)
    file.Upload()
    
    #delete local file
    if os.path.exists(file_name):
        os.remove(file_name)
    else:
        print("The file does not exist")